<a href="https://colab.research.google.com/github/joaocabeca2/ebac/blob/main/projeto1ebac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Análise Exploratória de Dados de Logística II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Manipulação;</li>
  <li>Visualização;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/).

---

# **Análise Exploratória de Dados de Logística**

## 1\. Contexto

In [1]:
!wget -q << EOF "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" EOF \-O deliveries.json

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')


In [2]:
import pandas as pd
import json
with open(file='deliveries.json',mode='r',encoding='utf8') as jsonfile:
    jsonfile = json.load(jsonfile)

In [3]:
deliveries_df = pd.DataFrame(jsonfile)
print(deliveries_df.info())
deliveries_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              199 non-null    object
 1   region            199 non-null    object
 2   origin            199 non-null    object
 3   vehicle_capacity  199 non-null    int64 
 4   deliveries        199 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.9+ KB
None


,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


In [4]:
origin_df = pd.json_normalize(deliveries_df.pop('origin'))
origin_df.head()

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145


In [5]:
deliveries_df = pd.merge(left=deliveries_df,right=origin_df,how='inner',right_index=True,left_index=True)
deliveries_df.head()

,name,region,vehicle_capacity,deliveries,lng,lat
0,cvrp-2-df-33,df-2,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


In [6]:
colunas_deliveries = list(deliveries_df['deliveries'][0][0].keys())
colunas_deliveries

['id', 'point', 'size']

In [7]:
deliveries_exploded = deliveries_df[['deliveries']].explode('deliveries')
deliveries_exploded.head()

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."


In [8]:
deliveries_atualizados = pd.concat([
      pd.DataFrame(deliveries_exploded['deliveries'].apply(lambda deliverie: deliverie['point']['lng'])).rename(columns={'deliveries':'deliveries_long'}),
      pd.DataFrame(deliveries_exploded['deliveries'].apply(lambda deliverie: deliverie['point']['lat'])).rename(columns={'deliveries':'deliveries_lat'}),
      pd.DataFrame(deliveries_exploded['deliveries'].apply(lambda deliverie: deliverie['size'])).rename(columns={'deliveries':'deliveries_size'}),
],axis=1)
deliveries_atualizados

,deliveries_long,deliveries_lat,deliveries_size
0,-48.116189,-15.848929,9
0,-48.118195,-15.850772,2
0,-48.112483,-15.847871,1
0,-48.118023,-15.846471,2
0,-48.114898,-15.858055,7
...,...,...,...
198,-48.064269,-15.997694,8
198,-48.065176,-16.003597,4
198,-48.065841,-16.003808,9
198,-48.062327,-16.001568,1


In [9]:
deliveries_df_atualizados = pd.merge(left=deliveries_df,right=deliveries_atualizados,how='right',left_index=True,right_index=True)
deliveries_df_atualizados.drop(['deliveries'],axis=1,inplace=True)
deliveries_df_atualizados.reset_index(inplace=True,drop=True)
deliveries_df_atualizados.head()

,name,region,vehicle_capacity,lng,lat,deliveries_long,deliveries_lat,deliveries_size
0,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.116189,-15.848929,9
1,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.118195,-15.850772,2
2,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.112483,-15.847871,1
3,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.118023,-15.846471,2
4,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.114898,-15.858055,7


In [10]:
print(deliveries_df_atualizados.shape)
print('\n',deliveries_df_atualizados.info())
print(deliveries_df_atualizados.isnull().any())


(636149, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636149 entries, 0 to 636148
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              636149 non-null  object 
 1   region            636149 non-null  object 
 2   vehicle_capacity  636149 non-null  int64  
 3   lng               636149 non-null  float64
 4   lat               636149 non-null  float64
 5   deliveries_long   636149 non-null  float64
 6   deliveries_lat    636149 non-null  float64
 7   deliveries_size   636149 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 38.8+ MB

 None
name                False
region              False
vehicle_capacity    False
lng                 False
lat                 False
deliveries_long     False
deliveries_lat      False
deliveries_size     False
dtype: bool


## 2\. Pacotes e bibliotecas

In [11]:
# importe todas as suas bibliotecas aqui, siga os padrões do PEP8:
#
# - 1º pacotes nativos do python: json, os, etc.;
# - 2º pacotes de terceiros: pandas, seabornm etc.;
# - 3º pacotes que você desenvolveu.
#

...

Ellipsis

## 3\. Exploração de dados

In [12]:
# faça o código de exploração de dados:
#
# - coleta de dados;
# - wrangling da estrutura;
# - exploração do schema;
# - etc.

## 4\. Manipulação

In [13]:
# faça o código de manipulação de dados:
#
# - enriquecimento;
# - controle de qualidade;
# - etc.

## 5\. Visualização

In [14]:
# faça o código de visualização de dados:
#
# - produza pelo menos duas visualizações;
# - adicione um pequeno texto com os insights encontrados;
# - etc.